# Import data& libraries 

In [171]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

In [172]:
path= "/mnt/d/Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
y = X_train["N_category"]
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

In [173]:
X_train.head(5)

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,63,2015-10-23,1,2,1,19.0,2.0,1.0,2.0,2.0,...,1.0,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN
1,51,2015-10-28,1,1,1,22.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN
2,37,2015-10-29,1,2,1,NaN,2.0,NaN,NaN,NaN,...,1.0,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0
3,54,2016-03-08,1,2,1,0.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN
4,57,2015-10-30,1,2,1,8.0,2.0,2.0,3.0,2.0,...,1.0,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN


In [174]:
print(X_train.info())
print(f"\nSize of X_train: {len(X_train)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   나이                 1000 non-null   int64  
 1   수술연월일              1000 non-null   object 
 2   진단명                1000 non-null   int64  
 3   암의 위치              1000 non-null   int64  
 4   암의 개수              1000 non-null   int64  
 5   암의 장경              931 non-null    float64
 6   NG                 949 non-null    float64
 7   HG                 914 non-null    float64
 8   HG_score_1         908 non-null    float64
 9   HG_score_2         908 non-null    float64
 10  HG_score_3         911 non-null    float64
 11  DCIS_or_LCIS_여부    1000 non-null   int64  
 12  DCIS_or_LCIS_type  126 non-null    float64
 13  T_category         996 non-null    float64
 14  ER                 999 non-null    float64
 15  ER_Allred_score    703 non-null    float64
 16  PR                 999 no

# Fill missing values

In [175]:
X_train = X_train.fillna(X_train.mean())

/tmp/ipykernel_2542/3112579809.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


# Process categorical values

In [176]:
X_train["수술연월일"].head(5)

0    2015-10-23
1    2015-10-28
2    2015-10-29
3    2016-03-08
4    2015-10-30
Name: 수술연월일, dtype: object

In [177]:
X_train["수술연월일"]

0      2015-10-23
1      2015-10-28
2      2015-10-29
3      2016-03-08
4      2015-10-30
          ...    
995    2006-12-22
996    2006-11-17
997    2006-11-10
998    2006-09-22
999    2010-09-01
Name: 수술연월일, Length: 1000, dtype: object

In [178]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

In [179]:
X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name

In [180]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   나이                 1000 non-null   int64         
 1   수술연월일              1000 non-null   datetime64[ns]
 2   진단명                1000 non-null   int64         
 3   암의 위치              1000 non-null   int64         
 4   암의 개수              1000 non-null   int64         
 5   암의 장경              1000 non-null   float64       
 6   NG                 1000 non-null   float64       
 7   HG                 1000 non-null   float64       
 8   HG_score_1         1000 non-null   float64       
 9   HG_score_2         1000 non-null   float64       
 10  HG_score_3         1000 non-null   float64       
 11  DCIS_or_LCIS_여부    1000 non-null   int64         
 12  DCIS_or_LCIS_type  1000 non-null   float64       
 13  T_category         1000 non-null   float64       
 14  ER       

In [181]:
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

In [182]:
X_train["수술연월일"] = date_list
X_train = X_train[X_train.columns[:-1]]

추후 위 코드 제거하고 요일 변수 전처리할것

In [183]:
from sklearn.model_selection import train_test_split

In [184]:
original_X = X_train.copy()

In [128]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [129]:
from sklearn.ensemble import RandomForestClassifier

In [130]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [131]:
model.score(X_valid, y_valid)

0.805

In [132]:
X_test

,ID,img_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,BC_01_0011,./test_imgs/BC_01_0011.png,55,2015-11-17,2,2,1,23.0,2.0,2.0,...,1.0,8.0,0.0,2.0,5.00,0.0,1.0,NaN,NaN,NaN
1,BC_01_0220,./test_imgs/BC_01_0220.png,43,2020-06-09,4,2,1,13.0,3.0,2.0,...,1.0,4.0,1.0,8.0,8.67,0.0,0.0,NaN,NaN,NaN
2,BC_01_0233,./test_imgs/BC_01_0233.png,76,2020-05-14,1,1,1,NaN,NaN,NaN,...,1.0,6.0,1.0,6.0,NaN,NaN,2.0,NaN,NaN,NaN
3,BC_01_0258,./test_imgs/BC_01_0258.png,58,2020-05-20,1,2,1,1.3,2.0,2.0,...,1.0,7.0,0.0,NaN,21.17,1.0,3.0,NaN,NaN,0.0
4,BC_01_0260,./test_imgs/BC_01_0260.png,56,2020-05-20,1,2,2,15.0,3.0,3.0,...,1.0,8.0,1.0,3.0,20.57,1.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,BC_01_3328,./test_imgs/BC_01_3328.png,61,2009-10-30,1,1,1,18.0,2.0,2.0,...,1.0,5.0,1.0,3.0,NaN,0.0,0.0,NaN,NaN,NaN
246,BC_01_3404,./test_imgs/BC_01_3404.png,42,2009-05-19,1,2,1,20.0,1.0,2.0,...,1.0,5.0,1.0,5.0,NaN,0.0,1.0,NaN,NaN,NaN
247,BC_01_3418,./test_imgs/BC_01_3418.png,37,2009-04-24,1,1,1,17.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN
248,BC_01_3438,./test_imgs/BC_01_3438.png,37,2009-02-06,1,1,1,7.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN


# Data preprocessing 

In [221]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
for train_index, test_index in skf.split(X, y):
...     print("TRAIN:", train_index, "TEST:", test_index)
...     X_train, X_test = X[train_index], X[test_index]
...     y_train, y_test = y[train_index], y[test_index]
TRAIN: [1 3] TEST: [0 2]

In [253]:
def process_score(X):
#     X_train, X_valid, y_train, y_valid = train_test_split(data, y, stratify=y, test_size=0.2)
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X, y)    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = RandomForestClassifier()
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        
        print(precision_recall_fscore_support(pred, y_test, average="macro"))

In [254]:
process_score(original_X)

(0.7297919167667067, 0.8194270616047837, 0.713319810682894, None)
(0.7446702031828646, 0.7832232649750899, 0.7395833333333334, None)
(0.670703633269943, 0.7827889238932184, 0.640893277971047, None)
(0.28385546992293065, 0.2670693560565203, 0.2694805194805195, None)
(0.5, 0.2425, 0.3265993265993266, None)


/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [151]:
feature_list=[]
for i in range(len(model.feature_importances_)):
    feature_list.append([X_train.columns[i], model.feature_importances_[i]])
feature_df = pd.DataFrame(feature_list)
feature_df.sort_values(1, ascending=False)
# feature_list = np.array(feature_list)
# plt.xticks(rotation=45)
# plt.bar(feature_list[:,0], feature_list[:,1].astype("float"))

,0,1
1,수술연월일,0.148001
24,year,0.107499
5,암의 장경,0.084935
6,NG,0.084690
11,DCIS_or_LCIS_여부,0.059004
0,나이,0.050954
26,day,0.048750
18,KI-67_LI_percent,0.044045
7,HG,0.038841
9,HG_score_2,0.033144


In [146]:
X = pd.read_csv(os.path.join(path, "train.csv"))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1000 non-null   object 
 1   img_path           1000 non-null   object 
 2   mask_path          1000 non-null   object 
 3   나이                 1000 non-null   int64  
 4   수술연월일              1000 non-null   object 
 5   진단명                1000 non-null   int64  
 6   암의 위치              1000 non-null   int64  
 7   암의 개수              1000 non-null   int64  
 8   암의 장경              931 non-null    float64
 9   NG                 949 non-null    float64
 10  HG                 914 non-null    float64
 11  HG_score_1         908 non-null    float64
 12  HG_score_2         908 non-null    float64
 13  HG_score_3         911 non-null    float64
 14  DCIS_or_LCIS_여부    1000 non-null   int64  
 15  DCIS_or_LCIS_type  126 non-null    float64
 16  T_category         996 no

In [140]:
corr = X.corr()

/tmp/ipykernel_2542/3921787701.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = X.corr()


In [144]:
corr["암의 장경"].sort_values(ascending=False)

암의 장경                1.000000
T_category           0.809469
HG                   0.353011
HER2_SISH_ratio      0.348833
NG                   0.311455
HG_score_2           0.309582
HG_score_3           0.308494
N_category           0.292102
HG_score_1           0.249808
KI-67_LI_percent     0.237337
나이                   0.127918
HER2_SISH            0.112631
DCIS_or_LCIS_type    0.095926
ER_Allred_score      0.080298
PR_Allred_score      0.062952
DCIS_or_LCIS_여부      0.021600
암의 개수                0.011457
진단명                  0.006628
HER2                -0.002667
암의 위치               -0.005704
HER2_IHC            -0.015271
BRCA_mutation       -0.078003
PR                  -0.101581
ER                  -0.140662
Name: 암의 장경, dtype: float64

암의 장경은 T_category및 HG와 연관성이 크므로 이를 이용하여 결측치를 채움

In [159]:
X.groupby("T_category").agg(["mean", "median"])["암의 장경"]

/tmp/ipykernel_2542/142092162.py:1: FutureWarning: ['ID', 'img_path', 'mask_path', '수술연월일'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  X.groupby("T_category").agg(["mean", "median"])["암의 장경"]


,mean,median
T_category,,
0.0,0.000000,0.0
1.0,12.192583,13.0
2.0,27.715082,25.0
3.0,66.812500,60.0
4.0,60.750000,68.0


In [227]:
process_score(original_X)

KeyError: "None of [Int64Index([178, 179, 180, 181, 182, 183, 184, 185, 186, 187,\n            ...\n            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],\n           dtype='int64', length=800)] are in the [columns]"

In [223]:
X["암의 장경"] = X["암의 장경"].fillna(X.groupby("T_category")["암의 장경"].transform("mean"))
temp_X = original_X.copy()
temp_X["암의 장경"] = X["암의 장경"]
process_score(temp_X)

KeyError: "None of [Int64Index([178, 179, 180, 181, 182, 183, 184, 185, 186, 187,\n            ...\n            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],\n           dtype='int64', length=800)] are in the [columns]"